# Featurized Dataset Creation

The purpose of this notebook is to craft a series of queries that will create "featurized" datasets for use in building a propensity model. In a previous data exploration notebook, we found that various eventType features had different levels of correlation with the Target variable that was a `web.formFilledOut` event. 

In this notebook, we will build *features* related to these `eventType` attributes, and create a flat **featurized** dataset for the purpose of egress to a Cloud environment.

This notebook is step 3 in the diagram below.

![exploration](../media/CME-PoC-Featurization.png)

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def get_result_as_df(cur):
    data = [r for r in cur]
    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(data, columns=colnames)

### Create the connection to the AEP Query Service

In [2]:


conn = psycopg2.connect("sslmode=require host=targetpremiumqa2.platform-query-stage.adobe.io port=80 dbname=prod:all? user=SECRETS")

### Begin building the query

In [3]:
%%time
cur = conn.cursor()
cur.execute('''SELECT * FROM analytics_experience_events_v2_synthetic LIMIT 5''')

CPU times: user 1.76 ms, sys: 2.5 ms, total: 4.26 ms
Wall time: 17.3 s


In [4]:
get_result_as_df(cur)

,_experience,_id,advertising,application,channel,commerce,dataSource,device,directMarketing,endUserIDs,...,marketing,media,placeContext,producedBy,receivedTimestamp,search,segmentMembership,timestamp,userActivityRegion,web
0,"(""(""(""(gGj7CnJj4,th8CK5,jm8ZWJ,RhqvjwY,4j2mDH,...",169a8b67-caf1-4188-bf26-a3678fa8d616,"(""(""(CU)"",1946c2f6-8e10-4a1d-a7f0-ae5b4be31a56...","(""(R7m8P2,43.84295652370955)"",close,""(LrI,8.26...","(ffb5b48d-aa9c-4eb2-82fe-c0d0be5332de,https://...","(""(LblUXb6,AiFd75Vj)"",""(BtSwuBzy,82.6832161951...","(a87d5951-5b02-4563-9db7-cf590ef06c2f,vbdqt)","(4,f,f,f,UNPROMPTED,01PKgzAxOz,N9FDI82sz,3aPD9...","(""(EIkdCY,66.2263045577043)"",""(o3Ny,27.7504959...","(""(""(loggedOut,td0ERksJyj,""(iEb51xRV)"",f,yzyZO...",...,"(5H6aoST,ogTyC8FudY,us)","(""(""(Y0YK,12.21256938224355)"",""(VCbMQ,80.16298...","(""(""(zc,""(6.040501274081844,14.873362462130643...",system,2022-09-11 00:01:45.397,"(f,l3Vv,0,18,3F1lACjjyE,https://www.nxTmOVNxyf...",None,2022-09-11 00:01:45.397,"(2022-09-11 00:01:45.397,CA14)","(""(Ye,""(VaLE0,55.58321407531465)"",LWrB3Y,Cs1TC..."
1,"(""(""(""(9W,3U9,7qlB1ZJcSX,uoRtDZCw8,WGcr,sl8P5X...",116640ff-6ddd-4912-b5a2-cb54177a2405,"(""(""(dRg4jlO)"",1e67e568-68a9-468e-b13a-c52feaa...","(""(1xLU,29.774346518508217)"",close,""(bN6cS9gYU...","(9af548f6-6bea-496e-853e-aa01014393e1,https://...","(""(4I1BDdfg,khIRhLkD)"",""(GLwaxRa,32.7363081105...","(7bb62b8d-e271-4333-aa18-6b7442858901,j62Iz0qN)","(0,f,f,f,ALWAYS_ALLOWED,xqbycY,de,qDZrT16,12,p...","(""(gfB8f6tF,28.012956005999534)"",""(ZSOXiNFk,30...","(""(""(authenticated,3RPn,""(oELO)"",f,SOYGK6iXwB)...",...,"(acFEEgt,TSM,3riD)","(""(""(jH,82.65047749724675)"",""(LWT9PsHlX,19.644...","(""(""(YZXOGRO7ID,""(18.642148347841935,65.695220...",salesRef,2022-09-11 00:08:49.397,"(f,mMShU,9,3,kmZzMDxQOn,https://www.I0sAq3ZIGs...",None,2022-09-11 00:08:49.397,"(2022-09-11 00:08:49.397,CAN2)","(""(GdshB8o7Rq,""(0YM,82.7718943913963)"",FJ0EeFF..."
2,"(""(""(""(EfDCw9DY,sd13YdO,t7fodVSw,7zrmaI2qLI,yL...",73728233-afa9-484f-9929-60c22281ee61,"(""(""(sfEra)"",d3a7fb44-af06-4dd6-a559-dd0a89a23...","(""(7EV1,87.38480630078082)"",close,""(055xlfo4a,...","(ced3115a-7edb-4320-bf48-a7414d311696,https://...","(""(DX,9BYGUOWqa)"",""(3s3wuEdd,63.64235620236237...","(2cd80d1f-22a2-4b90-887c-bf3a5e6de8b5,i7O6)","(2,f,f,f,ALWAYS_ALLOWED,umUm2p,2c5n2Q,7zUwrs,2...","(""(otrIZesx,41.647704247223736)"",""(BNM0dUK,18....","(""(""(ambiguous,mBnmg,""(Y9zfC2KO)"",f,Cq)"",""(amb...",...,"(NANkc4r,QJb43I,ClQq8AB3z)","(""(""(hCOyfQzG,4.2819718911712705)"",""(isN7Z,33....","(""(""(QZz,""(60.21387905591826,24.10935815476955...",system,2022-09-11 00:10:26.147,"(f,U5bYnR,17,12,T8zdIOa3y,https://www.80HF60fM...",None,2022-09-11 00:10:26.147,"(2022-09-11 00:10:26.147,IA1)","(""(0zPvMIK,""(24p,1.0)"",xV2B3otV,XyWummg,downlo..."
3,"(""(""(""(olwLJUp,5EjZofzIyZ,2tzyuhkF,1jB,Sv1DH2p...",5eeea476-d906-4921-984b-8984d7ec448d,"(""(""(f9)"",cd49c1a3-5a72-417e-b9b0-c70045b7ed78...","(""(LiYIfyR,88.76310971536492)"",close,""(PkUisW,...","(a823062a-19a9-403c-b404-d8e42b854633,https://...","(""(LYey,mWtH)"",""(GrycAI9e81,88.83436497967749)...","(377ddc0e-8a80-43d4-ade1-9cb775a56047,f566yKxNyc)","(8,f,f,f,UNPROMPTED,yLd,xM,XyTrPDs0uV,10,portr...","(""(TaXrZ,11.815814379812101)"",""(q0wfAJRieE,69....","(""(""(ambiguous,echRKG1I,""(jyG)"",f,R9Be8)"",""(am...",...,"(vSe8CaFrA,GD0D8p,KWv)","(""(""(sIjwCCULNG,88.64579294924813)"",""(qYCO,8.6...","(""(""(24hS6Q,""(38.66579202900298,10.91985370500...",system,2022-09-11 00:11:54.397,"(f,1JpEk2m,6,18,S8kER,https://www.EJDETmZFTd.c...",None,2022-09-11 00:11:54.397,"(2022-09-11 00:11:54.397,IRL2)","(""(ef7saT6cO,""(jbRF36ecP0,16.892836466578924)""..."
4,"(""(""(""(uRoCN,r2MDdiJ,LTGP4b,5pDIh2xM,Bg,h0p,m6...",c887ebfd-2d6f-4ed4-b705-f7654b3e5b4e,"(""(""(Z05dGhY)"",a71ce4ab-ad22-4fe8-bb04-2d3ae6e...","(""(8UeA,11.823396669738402)"",unknown,""(FYDCeqt...","(8644b538-d406-41ea-83dc-25c9057f839e,https://...","(""(Ngh14cjx,aY8PQUA7)"",""(UYj9aag9,24.792770510...","(f15a766

In [5]:
%%time
cur = conn.cursor()
cur.execute('''SELECT identityMap['ECID'][0].id, 
                      eventType,
                      timestamp
                FROM analytics_experience_events_v2_synthetic 
                LIMIT 5''')

CPU times: user 2.23 ms, sys: 366 µs, total: 2.59 ms
Wall time: 4.39 s


In [6]:
get_result_as_df(cur)

,identityMap[ECID][0].id,eventType,timestamp
0,14214076848494217430121537143309454037,web.webpagedetails.pageViews,2022-09-11 00:01:45.397
1,69180043791708382289214923436384851953,directMarketing.emailSent,2022-09-11 00:08:49.397
2,14214076848494217430121537143309454037,web.webinteraction.linkClicks,2022-09-11 00:10:26.147
3,25434470700682890430316704839617898227,directMarketing.emailSent,2022-09-11 00:11:54.397
4,14214076848494217430121537143309454037,web.webinteraction.linkClicks,2022-09-11 00:12:40.805


## Count distinct users in dataset

In [7]:
%%time
cur.execute('''SELECT COUNT(DISTINCT identityMap['ECID'][0].id)
                FROM analytics_experience_events_v2_synthetic 
                LIMIT 5''')
get_result_as_df(cur)

CPU times: user 3.43 ms, sys: 0 ns, total: 3.43 ms
Wall time: 11.2 s


,count(DISTINCT identityMap[ECID][0].id)
0,9991


## Look at sample metadata

In [8]:
%%time
cur.execute("SELECT sample_meta('multichannel_experience_event_dataset_v2')")

meta = get_result_as_df(cur)
meta

CPU times: user 3.23 ms, sys: 477 µs, total: 3.71 ms
Wall time: 297 ms


,sample_table_name,sample_dataset_id,parent_dataset_id,sample_type,sampling_rate,sample_num_rows,created
0,multichannel_experience_event_dataset_v2_sampl...,63605e1945dd601b996bbcd4,635f6768639dac1b9d486389,uniform,0.01,2576835,31/10/2022 11:45:29
1,multichannel_experience_event_dataset_v2_sampl...,636066124e84c61b997e1823,635f6768639dac1b9d486389,uniform,0.10,25772100,01/11/2022 12:19:30


In [9]:
meta["sample_table_name"].values

array(['multichannel_experience_event_dataset_v2_sample_0_01',
       'multichannel_experience_event_dataset_v2_sample_0_1'],
      dtype=object)

### Window functions with relative time

A very commonly used ML feature is "aggregate of some feature in the last {calendar window}

<div class="alert alert-warning">
    <ul>
        <li> I cannot seem to get window functions with relative timestamps to work from this jupyter client. 
            Note, they are available in Spark.</li>
        <li> I also cannot get a simple syntax of window functions to work
        </li>
    </ul>
    
</div>

In [10]:
cur.execute('''SELECT identityMap['ECID'][0].id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN 2 days PRECEDING AND CURRENT ROW) AS "emailsReceived"
FROM analytics_experience_events_v2_synthetic
LIMIT 50
                ''')

SyntaxError: ErrorCode: 42601 sessionId: 965670e3-4430-45b3-9376-e2fe6e39e3a2 Syntax error encountered. Reason: [line 5:91: extraneous input 'days' expecting {'.', '[', 'AND', 'AT', 'BETWEEN', 'FOLLOWING', ILIKE, 'IN', 'IS', 'LIKE', NILIKE, 'NOT', NRLIKE, 'OR', 'PRECEDING', RLIKE, 'OPERATOR', '=', NEQ, '<', '!<', '<=', '>', '!>', '>=', '==', '+', '-', '*', '/', '%', '||', '::', '^'}]


### Window functions over "unbounded preceeding"
The following query completes successfully. It computes the number of various event types over an unbounded window

In [11]:
%%time
cur.execute('''SELECT identityMap['ECID'][0].id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY randn()) AS random_row_number_for_user
           
FROM analytics_experience_events_v2_synthetic
LIMIT 50
                ''')

CPU times: user 3.2 ms, sys: 0 ns, total: 3.2 ms
Wall time: 9.93 s


In [12]:
get_result_as_df(cur).fillna(0)

,userId,eventType,timestamp,subscriptionOccurred,emailsReceived,emailsOpened,emailsClicked,productsViewed,propositionInteracts,propositionDismissed,webLinkClicks,minutes_since_emailSent,minutes_since_emailOpened,minutes_since_emailClick,minutes_since_linkClick,minutes_since_propositionInteract,minutes_since_propositionDismiss,minutes_since_linkClick,random_row_number_for_user
0,01062571734453589220284531148144342360,web.webinteraction.linkClicks,2022-09-06 06:24:56.094,0,0,0,0,1,0,0,2,0.0,0.0,0.0,20.0,0,0,0.0,1
1,01062571734453589220284531148144342360,advertising.impressions,2022-08-05 20:20:14.397,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,2
2,01062571734453589220284531148144342360,directMarketing.emailSent,2022-09-11 06:36:23.397,0,1,0,0,1,0,0,2,0.0,0.0,0.0,7232.0,0,0,7211.0,3
3,01062571734453589220284531148144342360,directMarketing.emailSent,2022-10-04 19:32:44.397,0,6,0,0,1,0,0,2,0.0,0.0,0.0,41128.0,0,0,41107.0,4
4,01062571734453589220284531148144342360,directMarketing.emailOpened,2022-10-06 19:30:13.249,0,6,1,0,1,0,0,2,2877.0,0.0,0.0,44005.0,0,0,43985.0,5
5,01062571734453589220284531148144342360,web.webpagedetails.pageViews,2022-08-19 06:17:24.397,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,6
6,01062571734453589220284531148144342360,commerce.productViews,2022-09-06 06:04:20.450,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,7
7,01062571734453589220284531148144342360,directMarketing.emailSent,2022-09-17 15:07:28.397,0,2,0,0,1,0,0,2,0.0,0.0,0.0,16383.0,0,0,16362.0,8
8,01062571734453589220284531148144342360,web.webpagedetails.pageViews,2022-09-06 05:58:27.397,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,9
9,01062571734453589220284531148144342360,web.webinteraction.linkClicks,2022-09-06 06:05:15.100,0,0,0,0,1,0,0,1,0.0,0.0,0.0,0.0,0,0,0.0,10


### Create Positive and Negative Labels

- Positive Labels are those people who subscribed - we select a feature row the time they subscribed.
- Negative Labels are those who never subscribed - for them, we select a random row (don't want to select the final row, as this may bias the negative labels vs. the positive ones)

In [13]:
%%time
cur.execute('''
CREATE OR REPLACE VIEW feature_data AS (
    SELECT identityMap['ECID'][0].id as userId, 
       eventType,
       timestamp,
       
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id) 
           AS "subscriptionOccurred",
       
       
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY randn()) AS random_row_number_for_user
    FROM analytics_experience_events_v2_synthetic
);

SELECT * 
FROM 
    (SELECT *
        FROM feature_data
        WHERE eventType = 'web.formFilledOut'
    UNION 
      (   SELECT *
            FROM feature_data f2
            WHERE f2.subscriptionOccurred = 0 AND f2.random_row_number_for_user = 1
    )
    )
ORDER BY timestamp
                ''')

CPU times: user 7.99 ms, sys: 11.7 ms, total: 19.7 ms
Wall time: 18.1 s


**Check that the final result contains as many distinct users as we started with**

In [14]:
final_result = get_result_as_df(cur)
print("Number unique users in featurized data: ",final_result["userId"].nunique())
print("Number of rows in featurized data: ",len(final_result))

Number unique users in featurized data:  9991
Number of rows in featurized data:  9991


In [15]:
final_result

,userId,eventType,timestamp,subscriptionOccurred,emailsReceived,emailsOpened,emailsClicked,productsViewed,propositionInteracts,propositionDismissed,webLinkClicks,minutes_since_emailSent,minutes_since_emailOpened,minutes_since_emailClick,minutes_since_productView,minutes_since_propositionInteract,minutes_since_propositionDismiss,minutes_since_linkClick,random_row_number_for_user
0,25576255760872394021862743814360451152,directMarketing.emailSent,2022-08-02 20:04:03.949,0,1,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,None,NaN,1
1,08534746960055684882075066038486291066,directMarketing.emailSent,2022-08-02 20:21:20.915,0,1,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,None,NaN,1
2,61756034867944784402792276997827737259,directMarketing.emailSent,2022-08-02 20:37:58.455,0,1,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,None,NaN,1
3,76991519001345398362524216033202392355,web.webpagedetails.pageViews,2022-08-02 20:51:45.397,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,None,NaN,1
4,24770507673888932257666303495157070934,directMarketing.emailSent,2022-08-02 21:04:06.738,0,1,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,None,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,87413796104528050991809652878648848240,directMarketing.emailOpened,2022-10-13 19:20:55.518,0,1,1,0,0,0,0,0,8119.0,0.0,NaN,NaN,NaN,None,NaN,1
9987,73340987857458015918142715120891839617,directMarketing.emailOpened,2022-10-13 22:25:22.906,0,7,4,1,0,0,0,0,5847.0,0.0,5186.0,NaN,NaN,None,NaN,1
9988,09357483895364560233762927357279822076,directMarketing.emailOpened,2022-10-14 04:38:12.270,0,8,4,0,0,0,0,0,8980.0,0.0,NaN,NaN,NaN,None,NaN,1
9989,68219486189160749412155414367324739091,web.formFilledOut,2022-10-14 05:43:22.069,1,4,2,1,0,0,0,0,5049.0,9.0,6.0,NaN,NaN,None,NaN,2


# Create the output table
In this section we will attempt to take the previous query, and manually create the table, directly on AEP. 
In previous tests, creating a "table" from the query service would then create a dataset directly on AEP. 


<div class="alert alert-warning">
    <ul>
        <li> This took ~ 5~10 minutes to complete for the 100K rows dataset, and 2.25 hours for the 27B row dataset </li>
        <li> Would be great to have some feedback/monitoring of these kinds of processes.</li>
    </ul>
    
</div>

**ONLY LAUNCH FOLLOWING IF YOU ARE PREPARED TO WAIT**

In [19]:
%%time
cur.execute('''
CREATE OR REPLACE VIEW feature_data AS (
    SELECT identityMap['ECID'][0].id as userId, 
       eventType,
       timestamp,
       
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id) 
           AS "subscriptionOccurred",
       
       
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY identityMap['ECID'][0].id ORDER BY randn()) AS random_row_number_for_user
    FROM analytics_experience_events_v2_synthetic
);




CREATE TABLE IF NOT EXISTS featurized_subscription_propensity_dataset;

INSERT INTO featurized_subscription_propensity_dataset (
    SELECT * 
    FROM 
        (SELECT *
            FROM feature_data
            WHERE eventType = 'web.formFilledOut'
        UNION 
          (   SELECT *
                FROM feature_data f2
                WHERE f2.subscriptionOccurred = 0 AND f2.random_row_number_for_user = 1
        )
        )
    ORDER BY timestamp
)

                ''')

CPU times: user 348 ms, sys: 122 ms, total: 470 ms
Wall time: 2h 23min 13s


## Validate the output table that we just wrote

This becomes buggy once we've written many times to the dataset. 
**ONLY DO THIS IF YOU ARE PREPARED TO WAIT**

In [145]:
cur.execute('''
SELECT * FROM featurized_subscription_propensity_dataset
''')

In [146]:
table_created = get_result_as_df(cur)
print("Number unique users: ",table_created["userId"].nunique())
print("Number of rows: ",len(table_created))

Number unique users:  9991
Number of rows:  9991
